Reduce overfiting of MobileNetV3_concrete_crack_detector

In [1]:
import os
import random
import time
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image
from collections import Counter

import matplotlib.pyplot as plt
import albumentations as A
from albumentations.pytorch import ToTensorV2

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import Dataset, DataLoader, random_split, WeightedRandomSampler
import torchvision.models as models
import torch.multiprocessing
torch.multiprocessing.set_start_method('spawn', force=True)

from torchvision import  models,transforms
from torchvision.models import resnet18, ResNet18_Weights, mobilenet_v3_small, mobilenet_v3_large, MobileNet_V3_Small_Weights, MobileNet_V3_Large_Weights

from sklearn.metrics import (accuracy_score, precision_recall_fscore_support, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix)

In [2]:
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed()

In [3]:
class ConcreteCrackDataset(Dataset):
  
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.label_map = {"Non-cracked": 0, "Cracked": 1}
    
    def __len__(self):
        return len(self.annotations)
    
    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.annotations.iloc[idx, 0])
        image = Image.open(img_name).convert("RGB")
        label_str = self.annotations.iloc[idx, 1]
        
        # Handle case sensitivity and strip any whitespace
        label_str = label_str.strip()
        if label_str.lower() == "non-cracked" or label_str.lower() == "non-crack" or label_str.lower() == "non-cracked":
            label = 0
        elif label_str.lower() == "cracked" or label_str.lower() == "crack":
            label = 1
        else:
            print(f"Unknown label: {label_str}, defaulting to Non-cracked (0)")
            label = 0
        
        if self.transform:
            image = self.transform(image)
        
        return image, label


In [4]:
# Load pretrained MobileNetV3 model
def get_MobileNetV3_model(num_classes=2):

    weights = MobileNet_V3_Large_Weights.DEFAULT
    model = mobilenet_v3_large(weights=weights)
    model.classifier[3] = nn.Linear(model.classifier[3].in_features, num_classes)
    
    return model

In [5]:
def evaluate(model, data_loader, device):
    model.eval()
    correct = 0
    total = 0
    all_preds = []
    all_targets = []
    
    with torch.no_grad():
        for images, targets in tqdm(data_loader, desc="Evaluating"):
            images = images.to(device)
            targets = targets.to(device)
            
            # Get predictions
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            
            # Update statistics
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
            
            # Store predictions and targets for additional metrics
            all_preds.extend(predicted.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())
    
    # Calculate accuracy
    accuracy = 100 * correct / total
    
    # Calculate confusion matrix
    cm = confusion_matrix(all_targets, all_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(all_targets, all_preds, average='binary')
    
    print(f"Accuracy: {accuracy:.2f}%")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-Score: {f1:.4f}")
    print(f"Confusion Matrix:\n{cm}")
    
    return {
        "accuracy": torch.tensor(accuracy),
        "precision": torch.tensor(precision),
        "recall": torch.tensor(recall),
        "f1": torch.tensor(f1)
    }

In [6]:
def train_one_epoch(model, data_loader, criterion, optimizer, device):
    model.train()
    
    total_loss = 0
    num_batches = 0
    correct = 0
    total = 0
    
    for images, targets in tqdm(data_loader, desc="Training"):
        try:
            # Move data to device
            images = images.to(device)
            targets = targets.to(device)
            
            # Zero the parameter gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, targets)
            
            # Backward pass
            loss.backward()
            
            # Clip gradients to prevent explosion
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            
            # Update weights
            optimizer.step()
            
            # Update statistics
            total_loss += loss.item()
            num_batches += 1
            
            # Calculate accuracy
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
            
            # Print loss occasionally
            if num_batches % 50 == 0:
                accuracy = 100 * correct / total
                print(f"Batch {num_batches}, Loss: {loss.item():.4f}, Accuracy: {accuracy:.2f}%")
                
        except Exception as e:
            print(f"Error in batch: {e}")
            continue
    
    # Calculate average loss and accuracy
    if num_batches == 0:
        print("WARNING: No valid batches in this epoch!")
        return 0.0, 0.0
    
    avg_loss = total_loss / num_batches
    accuracy = 100 * correct / total
    
    print(f"Processed {num_batches} batches")
    print(f"Training Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")
    
    return avg_loss, accuracy

In [7]:
# Function to train the model
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, device, num_epochs):
    best_accuracy = 0.0
    best_model_wts = None
    
    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        print("-" * 10)
        
        # Train for one epoch
        epoch_loss, train_accuracy = train_one_epoch(model, train_loader, criterion, optimizer, device)
        print(f"Training Loss: {epoch_loss:.4f}, Training Accuracy: {train_accuracy:.2f}%")
        
        # Update learning rate
        scheduler.step()
        
        # Evaluate on validation set
        val_metrics = evaluate(model, val_loader, device)
        val_accuracy = val_metrics['accuracy'].item()
        val_f1 = val_metrics['f1'].item()
        
        print(f"Validation Accuracy: {val_accuracy:.2f}%, F1-Score: {val_f1:.4f}")
        
        # Save best model based on accuracy
        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            best_model_wts = model.state_dict().copy()
            torch.save(best_model_wts, f'best_model_epoch_{epoch+1}.pth')
            print(f"Saved best model with Accuracy: {best_accuracy:.2f}%")
        
        print()
    
    # Load best model weights
    model.load_state_dict(best_model_wts)
    return model, best_accuracy


In [8]:
def main():
    
    # Set random seeds for reproducibility
    set_seed(42)
    
    # Set device
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    print(f"Using device: {device}")
    
    # Transforms
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # MobileNetV3 typically uses 224x224
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    train_dataset = ConcreteCrackDataset('../artifact_folder/train/labels.csv', '../artifact_folder/train/images', transform)
    val_dataset = ConcreteCrackDataset('../artifact_folder/val/labels.csv', '../artifact_folder/val/images', transform)
    test_dataset = ConcreteCrackDataset('../artifact_folder/test/labels.csv', '../artifact_folder/test/images', transform)
    
    # Check if datasets are loaded correctly
    print(f"Train dataset size: {len(train_dataset)}")
    print(f"Val dataset size: {len(val_dataset)}")
    print(f"Test dataset size: {len(test_dataset)}")
    print(f"----------------------------------------------------------------------------")
    
    # Verify a few samples to ensure labels are valid
    print("Checking a few samples from training set:")
    for i in range(min(3, len(train_dataset))):
        img, label = train_dataset[i]
        print(f"Sample {i} - Image shape: {img.shape}, Label: {label}")

    # Use a batch size suitable for MobileNetV3
    batch_size = 32
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)
    # Get MobileNetV3 model
    model = get_MobileNetV3_model(num_classes=2)  # 2 classes: Non-cracked (0) / Cracked (1)
    model.to(device)
    # Define loss function
    criterion = nn.CrossEntropyLoss()
    # Define optimizer and learning rate scheduler
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
    # Use learning rate scheduler to reduce lr by 0.1 every 3 epochs
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)
    print(f"----------------------------------------------------------------------------")
    # Train model
    print("Starting training...")
    start_time = time.time()
    model, best_accuracy = train_model(
        model,
        train_loader,
        val_loader,
        criterion,
        optimizer,
        scheduler,
        device,
        num_epochs=10)
    end_time = time.time()
    print(f"Training completed in {(end_time - start_time) / 60:.2f} minutes")
    print(f"Best validation accuracy: {best_accuracy:.2f}%")
    torch.save(model.state_dict(), 'Optimize_MobileNetV3_concrete_crack_detector.pth')
    print(f"----------------------------------------------------------------------------")
    


Run Entire Processes to get MobileNetV3_concrete_crack_detector

In [9]:
if __name__ == "__main__":
    main()

Using device: cpu
Train dataset size: 16968
Val dataset size: 8413
Test dataset size: 8415
----------------------------------------------------------------------------
Checking a few samples from training set:
Sample 0 - Image shape: torch.Size([3, 224, 224]), Label: 0
Sample 1 - Image shape: torch.Size([3, 224, 224]), Label: 0
Sample 2 - Image shape: torch.Size([3, 224, 224]), Label: 0
----------------------------------------------------------------------------
Starting training...
Epoch 1/10
----------


Training:   9%|▉         | 50/531 [02:45<25:46,  3.22s/it]

Batch 50, Loss: 0.8710, Accuracy: 72.88%


Training:  19%|█▉        | 100/531 [05:39<23:40,  3.29s/it]

Batch 100, Loss: 0.3337, Accuracy: 76.06%


...

Batch 500, Loss: 0.2029, Accuracy: 96.27%


Training: 100%|██████████| 531/531 [29:00<00:00,  3.28s/it]


Processed 531 batches
Training Loss: 0.1126, Accuracy: 96.29%
Training Loss: 0.1126, Training Accuracy: 96.29%


Evaluating: 100%|██████████| 263/263 [05:18<00:00,  1.21s/it]

Accuracy: 88.07%
Precision: 0.5601
Recall: 0.8746
F1-Score: 0.6829
Confusion Matrix:
[[6328  849]
 [ 155 1081]]
Validation Accuracy: 88.07%, F1-Score: 0.6829

Training completed in 351.28 minutes
Best validation accuracy: 93.91%
----------------------------------------------------------------------------


Evaluate Model using Test Set

In [11]:
# Load the model
print("Evaluating on test set...")

# Use the updated weights parameter instead of deprecated 'pretrained'
weights = MobileNet_V3_Large_Weights.DEFAULT
model = mobilenet_v3_large(weights=weights)
model.classifier[3] = nn.Linear(model.classifier[3].in_features, 2)  # 2 classes: cracked / not cracked

# Load trained weights
model.load_state_dict(torch.load('../model/Optimize_MobileNetV3_concrete_crack_detector.pth'))
model.eval()

# Define the transform for test data
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Evaluation function
def evaluate_model_on_split(split_name):
    base_path = f"../artifact_folder/{split_name}"
    labels_df = pd.read_csv(os.path.join(base_path, "labels.csv"))
    images_dir = os.path.join(base_path, "images")

    y_true, y_pred, y_score = [], [], []

    for _, row in labels_df.iterrows():
        img_path = os.path.join(images_dir, row["filename"])
        image = Image.open(img_path).convert("RGB")
        input_tensor = transform(image).unsqueeze(0).to(device)

        with torch.no_grad():
            output = model(input_tensor)
            pred = output.argmax(dim=1).item()
            prob = torch.softmax(output, dim=1)[0][1].item()

        label = 1 if row["label"].lower() == "cracked" else 0
        y_true.append(label)
        y_pred.append(pred)
        y_score.append(prob)

    return {
        "Accuracy": accuracy_score(y_true, y_pred),
        "Precision": precision_score(y_true, y_pred, zero_division=0),
        "Recall": recall_score(y_true, y_pred, zero_division=0),
        "F1-Score": f1_score(y_true, y_pred, zero_division=0),
        "AUC-ROC": roc_auc_score(y_true, y_score),
        "Confusion Matrix": confusion_matrix(y_true, y_pred).tolist()
    }

# Run on all sets
for split in ['train', 'val', 'test']:
    print(f"📊 Evaluation for {split.upper()}")
    metrics = evaluate_model_on_split(split)
    for k, v in metrics.items():
        print(f"{k}: {v}")
    print()
print(f"----------------------------------------------------------------------------")
print("Training and evaluation completed!")

Evaluating on test set...
📊 Evaluation for TRAIN
Accuracy: 0.9712989156058462
Precision: 0.9859035119698627
Recall: 0.9562706270627063
F1-Score: 0.9708610064022019
AUC-ROC: 0.9934172272804165
Confusion Matrix: [[8368, 116], [371, 8113]]

📊 Evaluation for VAL
Accuracy: 0.8806608819683822
Precision: 0.5601036269430052
Recall: 0.8745954692556634
F1-Score: 0.6828806064434618
AUC-ROC: 0.9540645391404492
Confusion Matrix: [[6328, 849], [155, 1081]]

📊 Evaluation for TEST
Accuracy: 0.8771241830065359
Precision: 0.5652392947103274
Recall: 0.8677494199535963
F1-Score: 0.6845637583892618
AUC-ROC: 0.9481623230785171
Confusion Matrix: [[6259, 863], [171, 1122]]

----------------------------------------------------------------------------
Training and evaluation completed!
